In [1]:
import io
import random
import string  # to process standard python strings
import warnings
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('popular', quiet=True)

# Uncomment the following only the first time
nltk.download('punkt')  # first-time use only
nltk.download('wordnet')  # first-time use only

# Reading in the corpus
with open('chatbot.txt', 'r', encoding='utf8', errors='ignore') as fin:
    raw = fin.read().lower()

# Tokenization
sent_tokens = nltk.sent_tokenize(raw)
word_tokens = nltk.word_tokenize(raw)

# Preprocessing
lemmer = WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

# Keyword Matching
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up", "hey",)
GREETING_RESPONSES = ["Hello, how are you feeling today?", "Hi there! How can I support you?", "Hello! I'm here to listen.", "Hi, what's on your mind?"]

def greeting(sentence):
    """If user's input is a greeting, return a greeting response"""
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

# Additional Responses for Mental Health Support
MENTAL_HEALTH_RESPONSES = {
    "how are you": "I'm here to help you. How are you feeling?",
    "i feel sad": "I'm sorry you're feeling this way. It's okay to feel sad sometimes. Do you want to talk about what's bothering you?",
    "i feel anxious": "Anxiety can be tough. Try to take deep breaths. Would you like to discuss what's making you anxious?",
    "i feel lonely": "Loneliness can be difficult. Remember that you're not alone. I'm here with you.",
    "i'm stressed": "Stress can be overwhelming. Have you tried taking a moment to relax? I'm here if you need to talk.",
    "i'm depressed": "I'm really sorry you're feeling this way. It's important to talk to someone who can help. Would you like some resources?",
    "i need help": "Reaching out is a brave step. How can I assist you?",
    "what should i do": "It's hard to know what to do sometimes. Talking about it might help. What's on your mind?",
    "bye": "Take care of yourself. I'm here whenever you need to talk.",
    "thank you": "You're welcome! I'm here for you anytime.",
    "can i talk to someone": "Talking to someone can really help. Would you like some suggestions on who you can reach out to?",
    "i feel hopeless": "I'm really sorry you're feeling this way. It might help to talk to someone you trust. You don't have to go through this alone.",
    "i feel angry": "It's okay to feel angry. Expressing your feelings can help. Do you want to talk about what's making you feel this way?",
    "i feel lost": "Feeling lost can be tough. Sometimes talking through it can help. I'm here to listen.",
    "can you help me": "I'm here to support you. How can I help?",
}

# Update the response function to include mental health responses
def response(user_response):
    robo_response = ''
    sent_tokens.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx = vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]

    if req_tfidf == 0:
        robo_response = "I’m sorry, I don’t quite understand. Could you tell me more?"
    else:
        robo_response = sent_tokens[idx]

    # Check if the user input has a mental health response
    if user_response in MENTAL_HEALTH_RESPONSES:
        robo_response = MENTAL_HEALTH_RESPONSES[user_response]

    return robo_response

# Chatbot conversation loop
flag = True
print("Julie: My name is Julie, and I'm here to listen and help. If you want to exit, type 'Bye'.")
while flag:
    user_response = input("You: ")
    user_response = user_response.lower()

    if user_response != 'bye':
        if user_response in ('thanks', 'thank you'):
            flag = False
            print("Julie: You're welcome. Take care of yourself.")
        else:
            if greeting(user_response) is not None:
                print("Julie: " + greeting(user_response))
            else:
                print("Julie: " + response(user_response))
                sent_tokens.remove(user_response)
    else:
        flag = False
        print("Julie: Goodbye! Take care, and remember I'm here whenever you need to talk.")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Julie: My name is Julie, and I'm here to listen and help. If you want to exit, type 'Bye'.
You: bye
Julie: Goodbye! Take care, and remember I'm here whenever you need to talk.
